In [2]:
from snyk_ai import Models
model = Models.Llama_3_2

# Advisories

> **Unstructured Data**: 8 security advisory markdown documents
   - Located in the `advisories/` directory
   - Each contains detailed vulnerability descriptions, code examples, attack vectors, remediation steps, and CVSS breakdown tables
   - Covers vulnerabilities like XSS, SQL Injection, RCE, CSRF, SSRF, Path Traversal, etc.


In [3]:
from pathlib import Path

advisories_dir = Path("../data/advisories").resolve()
advisory_files = sorted(advisories_dir.glob("*.md"))

print(f"Found {len(advisory_files)} advisory files:")
for f in advisory_files:
    print(f"  - {f.name}")

Found 8 advisory files:
  - advisory-001.md
  - advisory-002.md
  - advisory-003.md
  - advisory-004.md
  - advisory-005.md
  - advisory-006.md
  - advisory-007.md
  - advisory-008.md


In [4]:
from snyk_ai.utils.markdown import parse_markdown_document, BlockType

parsed = {}

for advisory_path in advisory_files:
    blocks = parse_markdown_document(advisory_path)
    
    print(f"{advisory_path.name}: {len(blocks)} blocks\n")

    assert blocks[0].type is BlockType.HEADER
    assert blocks[0].content.startswith("Security Advisory: ")
    print(f"  {blocks[0].content.removeprefix('Security Advisory: ')}\n")

    assert blocks[2].type is BlockType.HEADER
    assert blocks[2].content == "Executive Summary"
    assert blocks[3].type is BlockType.PARAGRAPH
    print(f"  {blocks[3].content}\n")

    parsed[advisory_path.name] = blocks

advisory-001.md: 34 blocks

  Cross-Site Scripting (XSS) in express-validator

  A critical Cross-Site Scripting (XSS) vulnerability has been discovered in the `express-validator` package affecting versions prior to 4.5.0. This vulnerability allows attackers to inject malicious JavaScript code through validation error messages, potentially compromising user sessions and sensitive data.

advisory-002.md: 37 blocks

  SQL Injection in webapp-auth

  A critical SQL injection vulnerability has been identified in the `webapp-auth` package affecting versions 2.0.0 through 2.3.0. This vulnerability allows attackers to execute arbitrary SQL commands through the authentication query builder, potentially leading to complete database compromise, data exfiltration, and authentication bypass.

advisory-003.md: 37 blocks

  Dependency Confusion in secure-config

  A dependency confusion vulnerability has been discovered in the `secure-config` package affecting versions 3.0.0 through 3.1.9. This vuln

In [18]:
advisory_3 = parsed["advisory-003.md"]
code_blocks = [b for b in advisory_3 if b.type is BlockType.CODE_BLOCK]
len(code_blocks)

for i, cb in enumerate(code_blocks):
    print(f"{i + 1}\n```{cb.language if cb.language else ''}\n{cb.content}\n")

1
```json
{
  "name": "my-application",
  "version": "1.0.0",
  "dependencies": {
    "secure-config": "^3.1.0"
  },
  "scripts": {
    "install": "npm install"
  }
}

2
```

3. When `npm install` runs, it may resolve to the public malicious package
4. The postinstall script executes automatically, exfiltrating sensitive data

### Impact

- **Confidentiality:** Sensitive configuration data, API keys, and secrets can be exfiltrated
- **Integrity:** Malicious code can modify application behavior or data
- **Availability:** Malicious packages can cause application failures or denial of service
- **Supply Chain:** Compromises the entire software supply chain

### CVSS Breakdown

| Metric | Value | Justification |
|--------|-------|---------------|
| Attack Vector | Network | Requires network access to package registry |
| Attack Complexity | Medium | Requires understanding of package resolution |
| Privileges Required | None | Public registry access is sufficient |
| User Interaction | Non

In [5]:
from snyk_ai.utils import summarize_document

# generate summaries for all advisories
summaries = {}
for advisory_path in advisory_files:
    print(f"Summarizing {advisory_path.name}...")
    summary_path = summarize_document(advisory_path, model)
    summaries[advisory_path.name] = summary_path.read_text()
    print(f"  -> Saved to {summary_path.name}")

print(f"\nCompleted {len(summaries)} summaries.")

Summarizing advisory-001.md...
  -> Saved to advisory-001-summary-ollama_llama3.2-20260123052642.txt
Summarizing advisory-002.md...
  -> Saved to advisory-002-summary-ollama_llama3.2-20260123052644.txt
Summarizing advisory-003.md...
  -> Saved to advisory-003-summary-ollama_llama3.2-20260123052646.txt
Summarizing advisory-004.md...
  -> Saved to advisory-004-summary-ollama_llama3.2-20260123052649.txt
Summarizing advisory-005.md...
  -> Saved to advisory-005-summary-ollama_llama3.2-20260123052651.txt
Summarizing advisory-006.md...
  -> Saved to advisory-006-summary-ollama_llama3.2-20260123052653.txt
Summarizing advisory-007.md...
  -> Saved to advisory-007-summary-ollama_llama3.2-20260123052656.txt
Summarizing advisory-008.md...
  -> Saved to advisory-008-summary-ollama_llama3.2-20260123052658.txt

Completed 8 summaries.


In [7]:
from snyk_ai.utils import summarize_code_snippet

# model = Models.Claude_Opus
print(f"### {model.name}\n")
for name, md_blocks in list(parsed.items()):
    code_blocks = [(b.content, b.language) for b in md_blocks if b.type is BlockType.CODE_BLOCK]
    print(f"#### {name} - {len(code_blocks)} code blocks\n")
    for i, (code, lang) in enumerate(code_blocks):
        summary = summarize_code_snippet(code, model)
        print(f"##### {i + 1}\n\n```{lang if lang else ''}\n{code}\n```\n\n> {summary}\n")

### ollama:llama3.2

#### advisory-001.md - 3 code blocks

##### 1

```javascript
const { body, validationResult } = require('express-validator');
const express = require('express');
const app = express();

app.post('/register', 
  body('email').isEmail(),
  body('username').isLength({ min: 3 }),
  (req, res) => {
    const errors = validationResult(req);
    if (!errors.isEmpty()) {
      // VULNERABLE: User input directly inserted into HTML
      return res.status(400).send(`
        <h1>Validation Error</h1>
        <p>${errors.array()[0].msg}</p>
        <p>Input: ${req.body.username}</p>
      `);
    }
    // ... registration logic
  }
);
```

> This code sets up an Express.js endpoint for registering new users, validating user input through middleware functions from the `express-validator` package before sending back an error response if validation fails.

##### 2

```
username=<script>alert(document.cookie)</script>
```

> The provided code injects malicious JavaScript that ale

### ollama:llama3.2

#### advisory-001.md - 3 code blocks

##### 1

```javascript
const { body, validationResult } = require('express-validator');
const express = require('express');
const app = express();

app.post('/register', 
  body('email').isEmail(),
  body('username').isLength({ min: 3 }),
  (req, res) => {
    const errors = validationResult(req);
    if (!errors.isEmpty()) {
      // VULNERABLE: User input directly inserted into HTML
      return res.status(400).send(`
        <h1>Validation Error</h1>
        <p>${errors.array()[0].msg}</p>
        <p>Input: ${req.body.username}</p>
      `);
    }
    // ... registration logic
  }
);
```

> This code sets up an Express.js endpoint for registering new users, validating user input through middleware functions from the `express-validator` package before sending back an error response if validation fails.

##### 2

```
username=<script>alert(document.cookie)</script>
```

> The provided code injects malicious JavaScript that alerts the contents of the current user's cookie.

##### 3

```javascript
const { body, validationResult } = require('express-validator');
const express = require('express');
const app = express();

// Helper function to escape HTML
function escapeHtml(text) {
  const map = {
    '&': '&amp;',
    '<': '&lt;',
    '>': '&gt;',
    '"': '&quot;',
    "'": '&#039;'
  };
  return text.replace(/[&<>"']/g, m => map[m]);
}

app.post('/register', 
  body('email').isEmail(),
  body('username').isLength({ min: 3 }),
  (req, res) => {
    const errors = validationResult(req);
    if (!errors.isEmpty()) {
      // FIXED: Properly escape user input
      const errorMsg = escapeHtml(errors.array()[0].msg);
      const username = escapeHtml(req.body.username);
      return res.status(400).send(`
        <h1>Validation Error</h1>
        <p>${errorMsg}</p>
        <p>Input: ${username}</p>
      `);
    }
    // ... registration logic
  }
);
```

> This Express.js code defines a POST endpoint for user registration that validates input using the `express-validator` library and displays error messages in a escaped HTML format if validation fails.

#### advisory-002.md - 3 code blocks

##### 1

```javascript
const db = require('./database');
const crypto = require('crypto');

class AuthService {
  async authenticateUser(username, password) {
    // VULNERABLE: Direct string concatenation in SQL query
    const query = `
      SELECT id, username, email, password_hash 
      FROM users 
      WHERE username = '${username}' 
      AND active = 1
    `;
    
    const result = await db.query(query);
    
    if (result.rows.length === 0) {
      throw new Error('Invalid credentials');
    }
    
    const user = result.rows[0];
    const passwordHash = crypto.createHash('sha256')
      .update(password + user.salt)
      .digest('hex');
    
    if (passwordHash !== user.password_hash) {
      throw new Error('Invalid credentials');
    }
    
    return user;
  }
}
```

> The provided code is an authentication service that takes a username and password as input, queries the database to verify the credentials, and returns a user object if the credentials are valid.

##### 2

```javascript
const db = require('./database');
const crypto = require('crypto');

class AuthService {
  async authenticateUser(username, password) {
    // FIXED: Use parameterized queries
    const query = `
      SELECT id, username, email, password_hash, salt
      FROM users 
      WHERE username = $1 
      AND active = 1
    `;
    
    // Input validation
    if (!username || typeof username !== 'string' || username.length > 100) {
      throw new Error('Invalid username format');
    }
    
    const result = await db.query(query, [username]);
    
    if (result.rows.length === 0) {
      // Use generic error message to prevent user enumeration
      throw new Error('Invalid credentials');
    }
    
    const user = result.rows[0];
    const passwordHash = crypto.createHash('sha256')
      .update(password + user.salt)
      .digest('hex');
    
    if (passwordHash !== user.password_hash) {
      throw new Error('Invalid credentials');
    }
    
    return user;
  }
}
```

> This code defines an `AuthService` class that authenticates a user by comparing their provided password to the hashed version stored in the database.

##### 3

```bash
# Test authentication endpoint
curl -X POST https://example.com/api/auth/login \
  -H "Content-Type: application/json" \
  -d '{"username": "admin'\'' OR '\''1'\''='\''1", "password": "test"}'
```

> This curl command sends a malicious POST request to an authentication endpoint attempting to bypass validation by exploiting a potential SQL injection vulnerability through the provided username.

#### advisory-003.md - 9 code blocks

##### 1

```json
{
  "name": "my-application",
  "version": "1.0.0",
  "dependencies": {
    "secure-config": "^3.1.0"
  },
  "scripts": {
    "install": "npm install"
  }
}
```

> This is the configuration of a JavaScript package (likely a Node.js project) specifying its name, version, dependencies, and an installation script.

##### 2

```

3. When `npm install` runs, it may resolve to the public malicious package
4. The postinstall script executes automatically, exfiltrating sensitive data

### Impact

- **Confidentiality:** Sensitive configuration data, API keys, and secrets can be exfiltrated
- **Integrity:** Malicious code can modify application behavior or data
- **Availability:** Malicious packages can cause application failures or denial of service
- **Supply Chain:** Compromises the entire software supply chain

### CVSS Breakdown

| Metric | Value | Justification |
|--------|-------|---------------|
| Attack Vector | Network | Requires network access to package registry |
| Attack Complexity | Medium | Requires understanding of package resolution |
| Privileges Required | None | Public registry access is sufficient |
| User Interaction | None | Automated installs can trigger |
| Scope | Changed | Can affect entire application |
| Confidentiality | High | Can exfiltrate sensitive data |
| Integrity | Medium | Can modify application behavior |
| Availability | Low | May cause application failures |

## Remediation

### Immediate Actions

1. **Upgrade immediately** to version 3.2.0 or later:
```

> When `npm install` runs, malicious packages can be installed and exfiltrate sensitive data through automatic execution of the postinstall script.

##### 3

```

2. **Use scoped packages** to prevent namespace conflicts:
```

> Using scoped packages helps prevent namespace conflicts by encapsulating packages within their own namespaces.

##### 4

```

3. **Implement package lock files** and verify checksums:
```

> The code instructs to implement package lock files and verify their checksums as part of ensuring reproducibility and integrity of software dependencies.

##### 5

```

### Fixed Configuration Example

```

> There is no code provided to summarize.

##### 6

```

### Additional Security Measures

1. **Use `.npmrc` Configuration:**
```

> This code section discusses using an `.npmrc` configuration file for additional security measures related to npm (Node Package Manager) authentication and permissions.

##### 7

```

2. **Implement Package Integrity Verification:**
```

> This section likely involves implementing checks to verify the integrity of packages during deployment or installation in a software system.

##### 8

```

3. **Use Private Package Registries:**
   - Configure npm to use private registries for internal packages
   - Set up proper authentication and authorization

4. **Implement CI/CD Security Checks:**
```

> Configure your project's package management and implement Continuous Integration/Continuous Deployment (CI/CD) security checks to protect against vulnerabilities in internal packages.

##### 9

```

5. **Monitor for Suspicious Packages:**
   - Set up alerts for new package versions
   - Review package contents before installation
   - Use tools like Snyk to detect dependency confusion

### Package Resolution Priority

| Registry Type | Priority | Risk Level |
|--------------|----------|------------|
| Private scoped registry | Highest | Low |
| Private unscoped registry | High | Medium |
| Public registry (scoped) | Medium | Low |
| Public registry (unscoped) | Lowest | High |

### Remediation Timeline

| Date | Action |
|------|--------|
| 2024-02-05 | Vulnerability discovered during supply chain audit |
| 2024-02-07 | Vendor notified |
| 2024-02-10 | CVE assigned and advisory published |
| 2024-02-10 | Fixed version 3.2.0 released |
| 2024-02-24 | Recommended patch deadline (14 days) |

## Best Practices for Preventing Dependency Confusion

1. **Use Scoped Packages:** Always use scoped package names for private packages (e.g., `@company/package-name`)
2. **Lock Dependencies:** Commit `package-lock.json` or `yarn.lock` to version control
3. **Verify Checksums:** Use `npm ci` which verifies package integrity
4. **Private Registries:** Host private packages in private registries with proper authentication
5. **Monitor Dependencies:** Regularly audit and update dependencies
6. **Least Privilege:** Run npm install with minimal required permissions
7. **CI/CD Security:** Implement security checks in build pipelines

## References

- [OWASP Dependency Confusion](https://owasp.org/www-community/vulnerabilities/Dependency_Confusion)
- [npm Package Scope Documentation](https://docs.npmjs.com/cli/v8/using-npm/scope)
- [Snyk Dependency Confusion Guide](https://snyk.io/blog/dependency-confusion-attacks/)
- [secure-config GitHub Repository](https://github.com/example/secure-config)
- [CVE-2024-1237 Details](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2024-1237)

## Credits

This vulnerability was discovered by the Snyk Security Research Team during a supply chain security assessment.


```

> This document outlines strategies for preventing and responding to dependency confusion attacks on software packages, including securing package dependencies, using private registries, and implementing best practices.

#### advisory-004.md - 2 code blocks

##### 1

```javascript
const fs = require('fs');
const path = require('path');

class DataProcessor {
  constructor(baseDir) {
    this.baseDir = baseDir;
  }

  async readFile(userPath) {
    // VULNERABLE: Direct path concatenation without validation
    const filePath = path.join(this.baseDir, userPath);
    const content = await fs.promises.readFile(filePath, 'utf8');
    return content;
  }
}

// Usage in API endpoint
app.get('/api/files/:filename', async (req, res) => {
  const processor = new DataProcessor('/app/uploads');
  const content = await processor.readFile(req.params.filename);
  res.send(content);
});
```

> This code defines a `DataProcessor` class to handle file reading and returns user-uploaded files directly from the server without any validation or sanitization.

##### 2

```javascript
class DataProcessor {
  constructor(baseDir) {
    this.baseDir = path.resolve(baseDir);
  }

  validatePath(userPath) {
    const normalizedPath = path.normalize(userPath);
    
    // Prevent directory traversal
    if (normalizedPath.includes('..')) {
      throw new Error('Invalid path: directory traversal detected');
    }
    
    const resolvedPath = path.resolve(this.baseDir, normalizedPath);
    
    // Ensure path is within base directory
    if (!resolvedPath.startsWith(this.baseDir)) {
      throw new Error('Invalid path: outside base directory');
    }
    
    return resolvedPath;
  }

  async readFile(userPath) {
    const filePath = this.validatePath(userPath);
    const stats = await fs.promises.stat(filePath);
    
    if (!stats.isFile()) {
      throw new Error('Path is not a file');
    }
    
    return await fs.promises.readFile(filePath, 'utf8');
  }
}
```

> This code defines a `DataProcessor` class that validates and processes file paths within a specified base directory, ensuring files are accessible and readable.

#### advisory-005.md - 2 code blocks

##### 1

```javascript
const fs = require('fs');

class FileHandler {
  async processFile(filePath) {
    const content = await fs.promises.readFile(filePath, 'utf8');
    
    // VULNERABLE: Unsafe deserialization using eval
    try {
      const data = eval(`(${content})`);
      return this.processData(data);
    } catch (error) {
      throw new Error('Invalid file format');
    }
  }

  async processSerializedData(serializedData) {
    // VULNERABLE: Using Function constructor (similar to eval)
    const deserializer = new Function('return ' + serializedData);
    const data = deserializer();
    return this.handleData(data);
  }
}
```

> This code provides a FileHandler class that attempts to deserialize JSON-like content from files using unsafe and vulnerable methods, including `eval` and the `Function` constructor.

##### 2

```javascript
class FileHandler {
  async processFile(filePath) {
    const content = await fs.promises.readFile(filePath, 'utf8');
    
    // Validate file size
    if (content.length > 10 * 1024 * 1024) {
      throw new Error('File too large');
    }
    
    // FIXED: Use safe JSON.parse instead of eval
    try {
      const data = JSON.parse(content);
      
      // Validate data structure
      if (!this.validateDataStructure(data)) {
        throw new Error('Invalid data structure');
      }
      
      return this.processData(data);
    } catch (error) {
      if (error instanceof SyntaxError) {
        throw new Error('Invalid JSON format');
      }
      throw error;
    }
  }

  validateDataStructure(data) {
    if (typeof data !== 'object' || data === null) {
      return false;
    }
    
    // Check for dangerous properties
    const dangerousProps = ['__proto__', 'constructor', 'prototype'];
    for (const prop of dangerousProps) {
      if (data.hasOwnProperty(prop)) {
        return false;
      }
    }
    
    return true;
  }
}
```

> This code defines a FileHandler class that processes and validates JSON files, throwing errors for invalid file sizes, data structures, or syntax.

#### advisory-006.md - 2 code blocks

##### 1

```python
from flask import Flask, request, session, jsonify
from api_client import APIClient

app = Flask(__name__)
app.secret_key = 'your-secret-key'

@app.route('/api/user/email', methods=['POST'])
def update_email():
    # VULNERABLE: No CSRF token validation
    if 'user_id' not in session:
        return jsonify({'error': 'Unauthorized'}), 401
    
    new_email = request.json.get('email')
    client = APIClient()
    result = client.update_user_email(session['user_id'], new_email)
    return jsonify({'success': True, 'email': new_email})
```

> This Flask API endpoint updates a user's email address, but lacks CSRF token validation and uses an external client to perform the actual update.

##### 2

```python
from flask import Flask, request, session, jsonify
from flask_wtf.csrf import CSRFProtect
from api_client import APIClient
import secrets

app = Flask(__name__)
app.secret_key = 'your-secret-key'
csrf = CSRFProtect(app)

def generate_csrf_token():
    """Generate CSRF token for session"""
    if 'csrf_token' not in session:
        session['csrf_token'] = secrets.token_hex(32)
    return session['csrf_token']

def validate_csrf_token():
    """Validate CSRF token from request"""
    token = request.headers.get('X-CSRF-Token') or request.form.get('csrf_token')
    if not token or token != session.get('csrf_token'):
        return False
    return True

@app.route('/api/user/email', methods=['POST'])
def update_email():
    if 'user_id' not in session:
        return jsonify({'error': 'Unauthorized'}), 401
    
    # FIXED: Validate CSRF token
    if not validate_csrf_token():
        return jsonify({'error': 'Invalid CSRF token'}), 403
    
    new_email = request.json.get('email')
    if not is_valid_email(new_email):
        return jsonify({'error': 'Invalid email format'}), 400
    
    client = APIClient()
    result = client.update_user_email(session['user_id'], new_email)
    return jsonify({'success': True, 'email': new_email})
```

> This Flask application validates user authentication and CSRF tokens before updating a user's email address through an API endpoint.

#### advisory-007.md - 2 code blocks

##### 1

```python
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

@app.route('/api/fetch', methods=['POST'])
def fetch_url():
    # VULNERABLE: No URL validation
    url = request.json.get('url')
    
    if not url:
        return jsonify({'error': 'URL required'}), 400
    
    # VULNERABLE: Direct request without validation
    try:
        response = requests.get(url, timeout=10)
        return jsonify({
            'status_code': response.status_code,
            'content': response.text[:1000]
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500
```

> This Flask API endpoint fetches the content of a provided URL and returns its status code and a limited excerpt of the response content in JSON format.

##### 2

```python
from urllib.parse import urlparse
import ipaddress
import socket

ALLOWED_DOMAINS = ['api.example.com', 'cdn.example.com']

def is_internal_ip(ip):
    """Check if IP is internal/private"""
    try:
        ip_obj = ipaddress.ip_address(ip)
        return ip_obj.is_private or ip_obj.is_loopback or ip_obj.is_link_local
    except ValueError:
        return False

def validate_url(url):
    """Validate URL to prevent SSRF"""
    if not url:
        return False, "URL is required"
    
    parsed = urlparse(url)
    
    # Only allow HTTP and HTTPS
    if parsed.scheme not in ['http', 'https']:
        return False, "Only HTTP and HTTPS protocols are allowed"
    
    hostname = parsed.hostname
    if not hostname:
        return False, "Invalid hostname"
    
    # Resolve and check IPs
    ips = socket.gethostbyname_ex(hostname)[2]
    for ip in ips:
        if is_internal_ip(ip):
            return False, "Internal IP addresses are not allowed"
    
    # Block cloud metadata endpoints
    blocked_hosts = ['169.254.169.254', 'metadata.google.internal']
    if hostname in blocked_hosts or any(ip in blocked_hosts for ip in ips):
        return False, "Cloud metadata endpoints are blocked"
    
    return True, None

@app.route('/api/fetch', methods=['POST'])
def fetch_url():
    url = request.json.get('url')
    is_valid, error_msg = validate_url(url)
    
    if not is_valid:
        return jsonify({'error': error_msg}), 400
    
    try:
        response = requests.get(url, timeout=10, allow_redirects=False)
        return jsonify({
            'status_code': response.status_code,
            'content': response.text[:1000]
        })
    except Exception as e:
        return jsonify({'error': 'Request failed'}), 500
```

> This code defines a Flask API endpoint that validates incoming URLs to prevent Server-Side Request Forgery (SSRF) attacks by checking the hostname and IP addresses against a list of allowed domains.

#### advisory-008.md - 2 code blocks

##### 1

```python
import json
import yaml
from json_parser import JSONParser

class DataProcessor:
    def process_json_data(self, json_string):
        # VULNERABLE: Using eval for "enhanced" JSON parsing
        try:
            data = eval(f"json.loads({repr(json_string)})")
            return self.handle_data(data)
        except:
            data = json.loads(json_string)
            return self.handle_data(data)
    
    def process_config(self, config_string):
        # VULNERABLE: Using yaml.load instead of yaml.safe_load
        config = yaml.load(config_string, Loader=yaml.Loader)
        return self.apply_config(config)
    
    def process_pickled_data(self, pickled_data):
        # VULNERABLE: Unpickling user-controlled data
        data = pickle.loads(pickled_data)
        return self.process_data(data)
```

> The provided code defines a DataProcessor class that processes different types of data (JSON, configuration files, and pickled data) but contains several security vulnerabilities due to improper use of eval() and yaml.load().

##### 2

```python
import json
import yaml

class DataProcessor:
    def process_json_data(self, json_string):
        # FIXED: Use safe JSON parsing only
        try:
            if len(json_string) > 10 * 1024 * 1024:  # 10MB limit
                raise ValueError("Input too large")
            
            # FIXED: Use standard json.loads (safe)
            data = json.loads(json_string)
            
            # Validate data structure
            if not self.validate_data_structure(data):
                raise ValueError("Invalid data structure")
            
            return self.handle_data(data)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON: {e}")
    
    def process_config(self, config_string):
        # FIXED: Use yaml.safe_load instead of yaml.load
        try:
            config = yaml.safe_load(config_string)
            
            if not isinstance(config, dict):
                raise ValueError("Config must be a dictionary")
            
            # Whitelist allowed configuration keys
            allowed_keys = ['timeout', 'retries', 'max_connections']
            filtered_config = {
                k: v for k, v in config.items() 
                if k in allowed_keys
            }
            
            return self.apply_config(filtered_config)
        except yaml.YAMLError as e:
            raise ValueError(f"Invalid YAML: {e}")
    
    def validate_data_structure(self, data):
        """Validate data structure to prevent object injection"""
        if not isinstance(data, (dict, list, str, int, float, bool, type(None))):
            return False
        
        # Check for dangerous keys
        dangerous_keys = ['__class__', '__init__', '__new__', '__reduce__']
        
        def check_dict(d):
            for key, value in d.items():
                if any(dk in str(key).lower() for dk in dangerous_keys):
                    return False
                if isinstance(value, dict):
                    if not check_dict(value):
                        return False
                elif isinstance(value, list):
                    if not check_list(value):
                        return False
            return True
        
        def check_list(l):
            for item in l:
                if isinstance(item, dict):
                    if not check_dict(item):
                        return False
                elif isinstance(item, list):
                    if not check_list(item):
                        return False
            return True
        
        if isinstance(data, dict):
            return check_dict(data)
        elif isinstance(data, list):
            return check_list(data)
        
        return True
```

> This class, `DataProcessor`, contains methods for safely parsing and validating JSON and YAML data, including checking for object injection and whitelisting specific configuration keys.

